12 柱状图

In [ ]:
#柱状图的开发

from pyecharts.charts import Bar
from pyecharts.options import*

#使用Bar构建基础柱状图
bar=Bar()
#添加x轴数据
bar.add_xaxis(["中国","美国","英国"])
#添加y轴数据
bar.add_yaxis("GDP",[30,20,10],label_opts=LabelOpts(position="right"))#数字标签放到右侧

#反转xy轴
bar.reversal_axis()
#数字标签放到右侧
#绘图
bar.render_notebook()

In [ ]:
#基础时间线柱状图绘制
#时间线：

from pyecharts.charts import Bar, Timeline
from pyecharts.options import*
from pyecharts.globals import ThemeType

bar1=Bar()
bar1.add_xaxis(["中国","美国","英国"])
bar1.add_yaxis("GDP",[30,30,20],label_opts=LabelOpts(position="right"))
bar1.reversal_axis()

bar2=Bar()
bar2.add_xaxis(["中国","美国","英国"])
bar2.add_yaxis("GDP",[50,50,50],label_opts=LabelOpts(position="right"))
bar2.reversal_axis()

bar3=Bar()
bar3.add_xaxis(["中国","美国","英国"])
bar3.add_yaxis("GDP",[70,60,60],label_opts=LabelOpts(position="right"))
bar3.reversal_axis()

#构建时间线对象
timeline=Timeline(
    {"theme":ThemeType.LIGHT} #变颜色
)
#在时间线内添加柱状图对象
timeline.add(bar1,"点1")
timeline.add(bar2,"点2")
timeline.add(bar3,"点3")


#自动播放设置
timeline.add_schema(
    play_interval=1000,#自动播放的时间毫秒
    is_timeline_show=True, #是否显示时间线
    is_auto_play=True, #是否自动播放
    is_loop_play=True #是否循环播放
)

#绘图是用时间线对象绘图，不是bar了
#timeline.render("基础时间线柱状图.html")
timeline.render_notebook()


### 列表的sort方法
使用方式：列表.sort(key=选择排序依据的函数，reverse=True（降序排序）\False（升序排序)  
         参数key：要求传入一个函数，表示将列表的每一个元素都传入函数中，返回排序的依据


In [2]:
#扩展列表的sort方法

#准备列表
my_list=[["a",33],["b",55],["c",11]]

#排序1，基于带名函数
def choose_sort_key(element):
    return element[1]

my_list.sort(key=choose_sort_key, reverse=True)
print(my_list)

#排序2，基于lambda函数
my_list.sort(key=lambda element:element[1],reverse=True)
print(my_list)

[['b', 55], ['a', 33], ['c', 11]]
[['b', 55], ['a', 33], ['c', 11]]


In [ ]:
#动态GDP柱状图绘制

from pyecharts.charts import Bar, Timeline
from pyecharts.options import*
from pyecharts.globals import ThemeType

#处理数据
f=open("/Users/miki/pythonlearn/12 数据可视化 动态柱状图/动态柱状图数据/1960-2019全球GDP数据.csv","r",encoding="GB2312")
data_lines=f.readlines()
#关闭文件
f.close()

#删除第一条数据
data_lines.pop(0)

#将数据转换为字典储存：格式为
#{ 年份:[ [国家，jdp]，[国家，jdp],...], 年份：[ [国家,jdp], [国家，jdp]，...],....}
#先定义一个字典对象
data_dict={}
for line in data_lines:
    #文件中都按逗号进行分割，用spilt方法
    year=int(line.split(",")[0])   #年份
    country=line.split(",")[1]      #国家
    gdp=float(line.split(",")[2])    #科学记数法转换为浮点数

    #如何判断字典里面有没有指定的key呢？异常判断
    try:
        data_dict[year].append([country,gdp])
    except KeyError:
        data_dict[year]=[]
        data_dict[year].append([country,gdp])
#print(data_dict[1960])

#创建时间线对象
timeline = Timeline({"theme":ThemeType.LIGHT}) #设置主题颜色
#排序年份
sorted_year_list=sorted(data_dict.keys())
for year in sorted_year_list:
    #用第二个数据进行降序排序
    data_dict[year].sort(key=lambda element:element[1],reverse=True)
    #取出本年份前八名的国家
    year_data=data_dict[year][1:8]
    x_data=[]
    y_data=[]
    for country_gdp in year_data:
        x_data.append(country_gdp[0])  #x轴添加国家
        y_data.append(country_gdp[1]/100000000)  #y轴添加gdp

    #构建柱状图对象
    bar=Bar()
    x_data.reverse()
    y_data.reverse()
    bar.add_xaxis(x_data)
    bar.add_yaxis("GDP(亿)",y_data,label_opts=LabelOpts(position="right"))
    #反转x轴和y轴
    bar.reversal_axis()
    #设置每一年图表的标题
    bar.set_global_opts(
        title_opts=TitleOpts(title=f"{year}年全球前8GDP数据")
    )

    timeline.add(bar,str(year))

#设置时间线自动播放
timeline.add_schema(
    play_interval=1000,
    is_timeline_show=True,
    is_auto_play=True,
    is_loop_play=False
)

#绘图
timeline.render_notebook()